In [139]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'fairbet_django.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [140]:
from order_placed.models import *
from django_pandas.io import read_frame

In [141]:
Betting.objects.filter(match__contains = "srl")

<BulkUpdateOrCreateQuerySet []>

In [142]:
df = read_frame(Betting.objects.filter(match__contains="Sr"))

In [143]:
df['winning_team'] = 'Srilanka'

In [144]:
winning_team = df['winning_team'].iloc[0]

In [145]:
def takeProfitLoss(row,winning_team="Srilanka"):
    if row.status == "BACK" and row.bet_on_team ==winning_team and row.winning_team == winning_team:
        return row.amount * row.odds - row.amount 
    elif row.status == "BACK" and row.bet_on_team == winning_team and row.bet_on_team != winning_team:
        return -row.amount
    elif row.status == "LAY" and row.bet_on_team == winning_team and row.bet_on_team == winning_team:
        return row.amount * - row.odds + row.amount
    elif row.status == "LAY" and row.bet_on_team == winning_team and row.bet_on_team != winning_team:
        return row.amount

In [146]:
df['L/P'] = df.apply(takeProfitLoss,axis=1)

In [147]:
df

,id,user,amount,bet_on_team,status,odds,winning_team,loss_profit,match,created_at,updated_at,L/P
0,7,fairbet,1000.0,Srilanka,BACK,1.85,Srilanka,0.0,Srilanka vs India,2022-07-04 07:05:52.327891+00:00,2022-07-04 07:05:52.331877+00:00,850.0
1,8,fairbet,1000.0,Srilanka,LAY,1.85,Srilanka,0.0,Srilanka vs India,2022-07-04 07:06:03.985142+00:00,2022-07-04 07:06:03.993118+00:00,-850.0


In [170]:
Betting.objects.filter(match = "Srilanka vs India").annotate(loss_profit=Case(When (status__eq="BACK",bet_on_team__eq="Srilanka",winning_team__eq="Not declared yet",then=0.0)))

ValueError: The annotation 'loss_profit' conflicts with a field on the model.

In [128]:
get_unique_match = df['match'].unique()

In [129]:
get_unique_match[0]

'Srilanka vs India'

In [106]:
df.groupby('match').get_group(get_unique_match[0])

,id,user,amount,bet_on_team,status,odds,winning_team,loss_profit,match,created_at,updated_at
0,7,fairbet,1000.0,Srilanka,BACK,1.85,Not declared yet,0.0,Srilanka vs India,2022-07-04 07:05:52.327891+00:00,2022-07-04 07:05:52.331877+00:00
1,8,fairbet,1000.0,Srilanka,LAY,1.85,Not declared yet,0.0,Srilanka vs India,2022-07-04 07:06:03.985142+00:00,2022-07-04 07:06:03.993118+00:00
